## Prophet Stock Model Training Notebook (Jupyter-ready)

In [4]:
import yfinance as yf
import pandas as pd
import joblib
import os
from prophet import Prophet
from sklearn.metrics import mean_absolute_error
from datetime import datetime


Importing plotly failed. Interactive plots will not work.


ModuleNotFoundError: No module named 'sklearn'